In [1]:
import urllib, urllib.request
import xmltodict
import json

'search_query' = 
 - ti (title):
 - au (author):
 - jr (journal reference):
 - cat (subject category):
 - rn (report number):
 - id (Id use id_list instead)
 - all (all of the above)

'start' and 'max_results' paging = 
 - Within the total results set, start defines the index of the first returned result, using 0-based indexing
 - max_results is the number of results returned by the query

sort order for return results = 
 - 'sortBy' = can be "relevance", "lastUpdatedDate", "submittedDate"
 - 'sortOrder' = can be either "ascending" or "descending"
 - "http://export.arxiv.org/api/query?search_query=ti:"electron thermal conductivity"&sortBy=lastUpdatedDate&sortOrder=ascending"

In [51]:
# arXiv category taxonomy
csCats = ['cs.AI', 'cs.CE', 'cs.DB', 'cs.ET', 'cs.DC', 'cs.LG', 'cs.IT']
statCats = ['stat.AP', 'stat.ML', 'stat.TH', 'stat.ME']
mathCats = ['math.PR', 'math.ST']
econCats = ['econ.EM']

In [75]:
def get_arxiv(db, category=list, file=bool):
    if type(category) != list:
        raise TypeError('Category is not in list format')
    else:
        for cat in category:
            url = 'http://export.arxiv.org/api/query?search_query=cat:{}&start=0&max_results=1000&sortBy=relevance&sortOrder=ascending'.format(cat)
            data = urllib.request.urlopen(url)
            arxiv_data = data.read().decode('utf-8')

            # Returned data is an "Atom Document" - convert to ordered dictionary
            arxiv_dict = xmltodict.parse(arxiv_data)

            # Converting to JSON
            arxivJSON = json.dumps(arxiv_dict, indent=4)

            # Decoding JSON
            arxiv_final = json.loads(arxivJSON)

            # Insert document into collection
            try:
                db.insert_many(arxiv_final['feed']['entry'])
            except:
                print('An error has occured')

            # Optional - write and save to file
            if file == True:
                with open('{}.json'.format(cat.replace('.', '_')), 'w') as write_file:
                    json.dump(arxiv_dict, write_file, indent=4)
            else:
                print('Data collection and insertion into MongoDB database is complete')

In [65]:
get_arxiv(econCats)